In [1]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.models import Span, CustomJS, Select, DateRangeSlider
from bokeh.models.annotations import Title
from bokeh.models import ColumnDataSource, HoverTool, ColorBar, FixedTicker, SingleIntervalTicker, LinearAxis
from bokeh.layouts import gridplot, column, row
from bokeh.transform import linear_cmap
from bokeh.palettes import all_palettes
import datetime
from selenium import webdriver
import chromedriver_binary
output_notebook()

Loading BokehJS ...

Creamos una sesion de `Spark`

In [2]:
spark = SparkSession.builder.appName('Practice').getOrCreate()

## Importamos la base de datos `ads_produccion.csv`

In [3]:
infer_schema = "True"
first_row_is_header = "True"
delimiter = ";"
file_type = "csv"
file_location = "ads_produccion.csv"

ads_produccion = (
   spark.read.format(file_type)
    .option("inferSchema", infer_schema)
    .option("header", first_row_is_header)
    .option("sep", ',')
    .load(file_location)
)

Variables que se encuentran en la base de datos

In [4]:
ads_produccion.printSchema()

root
 |-- Fecha_Hora: string (nullable = true)
 |-- ton_total: double (nullable = true)
 |-- n_descargas: double (nullable = true)
 |-- n_cam: double (nullable = true)
 |-- n_shov: double (nullable = true)
 |-- ton_chancador: double (nullable = true)
 |-- ton_botadero: double (nullable = true)
 |-- descargas_botadero: double (nullable = true)
 |-- ton_chancador_1: double (nullable = true)
 |-- ton_chancador_2: double (nullable = true)
 |-- cam_chancador: double (nullable = true)
 |-- cam_botadero: double (nullable = true)
 |-- ton_alta_ley: double (nullable = true)
 |-- ton_media_ley: double (nullable = true)
 |-- ton_baja_ley: double (nullable = true)
 |-- ton_lastre: double (nullable = true)
 |-- n_perfo: double (nullable = true)
 |-- n_eq_apoyo: double (nullable = true)
 |-- n_aljibe: double (nullable = true)



Cantidad de `records` en la base de datos

In [5]:
ads_produccion.count()

20676

### Chequeamos que no existan valores `null`

In [6]:
for col in ads_produccion.columns:
    nulls = ads_produccion.select(col).where(F.col(col).isNull()).count()
    print(f'Hay {nulls} valores NULL en la columna {col} de un total de {ads_produccion.count()}')

Hay 0 valores NULL en la columna Fecha_Hora de un total de 20676
Hay 130 valores NULL en la columna ton_total de un total de 20676
Hay 130 valores NULL en la columna n_descargas de un total de 20676
Hay 130 valores NULL en la columna n_cam de un total de 20676
Hay 130 valores NULL en la columna n_shov de un total de 20676
Hay 130 valores NULL en la columna ton_chancador de un total de 20676
Hay 130 valores NULL en la columna ton_botadero de un total de 20676
Hay 130 valores NULL en la columna descargas_botadero de un total de 20676
Hay 130 valores NULL en la columna ton_chancador_1 de un total de 20676
Hay 130 valores NULL en la columna ton_chancador_2 de un total de 20676
Hay 130 valores NULL en la columna cam_chancador de un total de 20676
Hay 130 valores NULL en la columna cam_botadero de un total de 20676
Hay 130 valores NULL en la columna ton_alta_ley de un total de 20676
Hay 130 valores NULL en la columna ton_media_ley de un total de 20676
Hay 130 valores NULL en la columna ton_b

### Chequeamos que no existan valores negativos o cero

In [7]:
for col in ads_produccion.columns:
    try:
        lt_zero = ads_produccion.select(col).where(F.col(col) < 0).count()
        print(f'Hay {lt_zero} valores negativos en la columna {col} de un total de {ads_produccion.count()}')
    except:
        print(f'La columna {col} no se le puede aplicar esta funcion ya que es de tipo {dict(ads_produccion.dtypes)[col]}')

Hay 0 valores negativos en la columna Fecha_Hora de un total de 20676
Hay 0 valores negativos en la columna ton_total de un total de 20676
Hay 0 valores negativos en la columna n_descargas de un total de 20676
Hay 0 valores negativos en la columna n_cam de un total de 20676
Hay 0 valores negativos en la columna n_shov de un total de 20676
Hay 0 valores negativos en la columna ton_chancador de un total de 20676
Hay 0 valores negativos en la columna ton_botadero de un total de 20676
Hay 0 valores negativos en la columna descargas_botadero de un total de 20676
Hay 0 valores negativos en la columna ton_chancador_1 de un total de 20676
Hay 0 valores negativos en la columna ton_chancador_2 de un total de 20676
Hay 0 valores negativos en la columna cam_chancador de un total de 20676
Hay 0 valores negativos en la columna cam_botadero de un total de 20676
Hay 0 valores negativos en la columna ton_alta_ley de un total de 20676
Hay 0 valores negativos en la columna ton_media_ley de un total de 20

### Filtramos la base de datos

In [8]:
ads_produccion = (
    ads_produccion.withColumn('fecha', F.to_timestamp('Fecha_Hora', 'yyyy-MM-dd HH:mm:ss'))
    .where((F.col('ton_total').isNotNull())
          & (F.col('ton_total') > 0))
    .drop('Fecha_Hora', 'n_perfo', 'n_eq_apoyo', 'n_aljibe')      
)

Cantidad de `records` en la base de datos luego del filtrado

In [9]:
ads_produccion.count()

20534

### Ewsta funcion permite visualizar los datos agrupados por hora de cualquier variable. Ademas podemos incluir informacion de otras variables para complementar el análisis

In [10]:
def plot_rango_fecha(df, x, y, date_column, fecha_inicio, x_label='', y_label='', extra_info=[], fecha_termino=None, span=True, cmap=True):
    """
    `df`  : pyspark DataFrame que contenga resultados de todas las variables agrupados por hora
    `x, y`: Nombre de columnas en `df` que se van a visualizar en el grafico
    `date_column`: Nombre de columna en `df` que contenga Hora
    `fecha_inicio`: fecha en formato yyyy-MM-dd que indica la fecha de inicio
    `fecha_inicio`: fecha en formato yyyy-MM-dd que indica la fecha de termoino, en caso de que sea None no se considera
    `x_label, `y_label`: Nombre de los ejes en el grafico
    `extra_info`: Lista que contenga nombres de variables que se quiera conocer la informacion
    `span`: Muestra una grilla que divide el grafico en 4, respecto del max y min de cada variable, default True.
    `cmap`: Incluir o no un color map como matix. Default True
    """
    if fecha_termino is None:
        df = (
            df.select('*')
            .where((F.year(F.col(date_column)) == fecha_inicio.split('-')[0])
                    & (F.month(F.col(date_column))== fecha_inicio.split('-')[1]) 
                    & (F.dayofmonth(F.col(date_column)) == fecha_inicio.split('-')[2]))
            .withColumn('h', F.hour(F.col(date_column)))).toPandas()
        title = f'Gráfico Comparativo {fecha_inicio}'
    else:
        df = (
            df.select('*')
            .filter(F.col(date_column).between(datetime.datetime.strptime(f'{fecha_inicio} 00:00:00', '%Y-%m-%d %H:%M:%S'),
                                               datetime.datetime.strptime(f'{fecha_termino} 23:00:00', '%Y-%m-%d %H:%M:%S')))
            .withColumn('h', F.hour(F.col(date_column)))).toPandas()
        title = f'Rango desde {fecha_inicio} a {fecha_termino}'
    x_plot = df[x]
    y_plot = df[y]

    data = {'x': x_plot, 'y': y_plot, 'date': df[date_column], 'h': df['h']}
    for idx, c in enumerate(extra_info):
        data[f'c{idx}'] = df[c]
    tooltips = [(date_column, '@date{%Y-%m-%d %H:%M}'), (y, '@y{int}'), (x, '@x{int}')] + [(f'{c}', f'@c{idx}') for idx, c in enumerate(extra_info)]
    source = ColumnDataSource(data=data)
    hover_tool = HoverTool(tooltips=tooltips, formatters={'@date': 'datetime'})
    
    mapper = linear_cmap(field_name='h', palette=all_palettes['PiYG'][4], low=0, high=23)
    color_bar = ColorBar(color_mapper=mapper['transform'], width = 20, ticker=FixedTicker(ticks=np.arange(0, 36, 6), desired_num_ticks=5), scale_alpha=0.7)
    
    p = figure(width=500, height=450, x_range=(x_plot.min(), x_plot.max()), y_range=(y_plot.min(), y_plot.max()))
    p.circle(x = 'x', y = 'y', size=10, color= (mapper if cmap else '#2171b5') , alpha=0.7, source=source)
    p.title.align = 'center'
    p.title.text = title
    p.title.text_font_size = '14pt'
    if (x_label == '' or y_label == ''):
        p.xaxis.axis_label = x
        p.yaxis.axis_label = y
    else:
        p.xaxis.axis_label = x_label
        p.yaxis.axis_label = y_label
        
    if span:
        vline = Span(location=(x_plot.max()+x_plot.min())*0.5, dimension='height', line_color='black', line_width=3, line_alpha=0.7)
        hline = Span(location=(y_plot.max()+y_plot.min())*0.5, dimension='width', line_color='black', line_width=3, line_alpha=0.7)
        p.renderers.extend([vline, hline])
    p.add_tools(hover_tool)
    if cmap:
        p.add_layout(color_bar, 'right')

    return p

#### Mostramos el tonelaje versus el numero de camiones, para una fecha en especifico. Los colores indican la hora del día

In [11]:
show(plot_rango_fecha(df=ads_produccion, date_column='fecha', x='n_cam', y='ton_total', x_label='Numero Camiones', y_label='Tonelaje total',
                      fecha_inicio='2021-11-14', span=True, fecha_termino=None, cmap=True))

#### Mostramos el tonelaje versus el numero de paladas. Se puede hacer con un rango de fechas, sin necesidad de incluir el `color map`

In [12]:
show(plot_rango_fecha(df=ads_produccion, date_column='fecha', x='n_shov', y='ton_total', fecha_inicio='2021-11-14', span=False, fecha_termino='2021-12-30', cmap=False))

#### Mostramos el tonelaje versus el numero de descargas para un rango de fechas

In [13]:
show(plot_rango_fecha(ads_produccion, date_column='fecha', x='n_descargas', y='ton_total', fecha_inicio='2021-11-14', span=False, fecha_termino='2021-12-14'))

## Importamos la base de datos `Datos_movimiento_mina.csv`

In [14]:
infer_schema = "True"
first_row_is_header = "True"
delimiter = ";"
file_type = "csv"
file_location = "Datos_movimiento_mina.csv"
movimiento_mina = (
   spark.read.format(file_type)
    .option("inferSchema", infer_schema)
    .option("header", first_row_is_header)
    .option("sep", delimiter)
    .load(file_location)
)

### Preprocesamiento de la base de datos

In [15]:
movimiento_mina = (
    movimiento_mina.select(
     F.col('TONELAJE FC').cast(T.DoubleType()),
     F.col('# Baldes').cast(T.DoubleType()),
     F.col('TIEMPO DE ACULATAMIENTO').cast(T.DoubleType()),
     F.col('TIEMPO DE CARGA').cast(T.DoubleType()),
     F.col('TIEMPO DE DESCARGA').cast(T.DoubleType()),
     F.col('TIEMPO ESPERA PALA').cast(T.DoubleType()),
     F.col('TIEMPO COLA CAMION').cast(T.DoubleType()),
     F.col('TIEMPO VIAJE HACIA LA PALA').cast(T.DoubleType()),
     F.col('TIEMPO DE VIAJE HACIA DESCARGA').cast(T.DoubleType()),
     F.col('TIEMPO DE CICLO TOTAL CAEX').cast(T.DoubleType()),
     F.col('TIEMPO DE CICLO EFECTIVO PALA').cast(T.DoubleType()),
     F.col('DISTANCIA DE VIAJE VACIO').cast(T.DoubleType()),
     F.col('DISTANCIA DE VIAJE LLENO').cast(T.DoubleType()),
     F.col('DISTANCIA EQUIVALENTE DE VIAJE VACIO').cast(T.DoubleType()),
     F.col('DISTANCIA EQUIVALENTE DE VIAJE LLENO').cast(T.DoubleType()),            
     'LoadingTimestamp',
     'DumpingTimestamp').withColumnRenamed('TIEMPO DE CARGA', 't_carga')
    .withColumnRenamed('TONELAJE FC', 'ton_camion')
    .withColumnRenamed('# Baldes', 'n_baldes')
    .withColumnRenamed('TIEMPO DE ACULATAMIENTO', 't_aculatamiento')
    .withColumnRenamed('TIEMPO DE DESCARGA', 't_descarga')
    .withColumnRenamed('TIEMPO ESPERA PALA', 't_espera_pala')
    .withColumnRenamed('TIEMPO COLA CAMION', 't_cola_camion')
    .withColumnRenamed('TIEMPO VIAJE HACIA LA PALA', 't_a_pala')
    .withColumnRenamed('TIEMPO DE VIAJE HACIA DESCARGA', 't_viaje_descarga')
    .withColumnRenamed('TIEMPO DE CICLO TOTAL CAEX', 't_ciclo_caex')
    .withColumnRenamed('TIEMPO DE CICLO EFECTIVO PALA', 't_ciclo_pala')
    .withColumnRenamed('DISTANCIA DE VIAJE VACIO', 'd_viaje_vacio')
    .withColumnRenamed('DISTANCIA DE VIAJE LLENO', 'd_viaje_lleno')
    .withColumnRenamed('DISTANCIA EQUIVALENTE DE VIAJE VACIO', 'd_eq_vacio')
    .withColumnRenamed('DISTANCIA EQUIVALENTE DE VIAJE LLENO', 'd_eq_lleno')
    .withColumn('Loading', F.to_timestamp('LoadingTimestamp', 'yyyy-MM-dd HH:mm:ss.SSS'))
    .withColumn('Dumping', F.to_timestamp('DumpingTimestamp', 'yyyy-MM-dd HH:mm:ss.SSS'))
    .drop('LoadingTimestamp', 'DumpingTimestamp')
)
movimiento_mina = movimiento_mina.select([F.when(F.col(c)=="NULL", None).otherwise(F.col(c)).alias(c) for c in movimiento_mina.columns])

Observamos la cantidad de variables que existen y el tipo de variable

In [16]:
movimiento_mina.printSchema()

root
 |-- ton_camion: double (nullable = true)
 |-- n_baldes: double (nullable = true)
 |-- t_aculatamiento: double (nullable = true)
 |-- t_carga: double (nullable = true)
 |-- t_descarga: double (nullable = true)
 |-- t_espera_pala: double (nullable = true)
 |-- t_cola_camion: double (nullable = true)
 |-- t_a_pala: double (nullable = true)
 |-- t_viaje_descarga: double (nullable = true)
 |-- t_ciclo_caex: double (nullable = true)
 |-- t_ciclo_pala: double (nullable = true)
 |-- d_viaje_vacio: double (nullable = true)
 |-- d_viaje_lleno: double (nullable = true)
 |-- d_eq_vacio: double (nullable = true)
 |-- d_eq_lleno: double (nullable = true)
 |-- Loading: timestamp (nullable = true)
 |-- Dumping: timestamp (nullable = true)



Cantidad de `records` en la base de datos

In [17]:
movimiento_mina.count()

731156

### Chequeamos que no existan valores `null` en la base de datos

In [18]:
for col in movimiento_mina.columns:
    nulls = movimiento_mina.select(col).where(F.col(col).isNull()).count()
    print(f'Hay {nulls} valores NULL en la columna {col} de un total de {movimiento_mina.count()}')

Hay 30485 valores NULL en la columna ton_camion de un total de 731156
Hay 6343 valores NULL en la columna n_baldes de un total de 731156
Hay 158901 valores NULL en la columna t_aculatamiento de un total de 731156
Hay 6344 valores NULL en la columna t_carga de un total de 731156
Hay 3 valores NULL en la columna t_descarga de un total de 731156
Hay 6344 valores NULL en la columna t_espera_pala de un total de 731156
Hay 6344 valores NULL en la columna t_cola_camion de un total de 731156
Hay 63797 valores NULL en la columna t_a_pala de un total de 731156
Hay 12437 valores NULL en la columna t_viaje_descarga de un total de 731156
Hay 6347 valores NULL en la columna t_ciclo_caex de un total de 731156
Hay 6344 valores NULL en la columna t_ciclo_pala de un total de 731156
Hay 30677 valores NULL en la columna d_viaje_vacio de un total de 731156
Hay 21408 valores NULL en la columna d_viaje_lleno de un total de 731156
Hay 49172 valores NULL en la columna d_eq_vacio de un total de 731156
Hay 83069

Hay mas de 158k datos `null` en la columna `t_aculatamiento`, sin contar otras variables que tienen una gran cantidad de valores `null` como `t_a_pala`. Todas estas columnas seran removidas de manera momentanea hasta que exista una actualizacion de la base de datos

### Chequeamos que no existan valores negativos en la base de datos

In [19]:
for col in movimiento_mina.columns:
    try:
        lt_zero = movimiento_mina.select(col).where(F.col(col) <= 0).count()
        print(f'Hay {lt_zero} valores negativos o zero en la columna {col} de un total de {movimiento_mina.count()}')
    except:
        print(f'La columna {col} no se le puede aplicar esta funcion ya que es de tipo {dict(movimiento_mina.dtypes)[col]}')

Hay 86 valores negativos o zero en la columna ton_camion de un total de 731156
Hay 0 valores negativos o zero en la columna n_baldes de un total de 731156
Hay 0 valores negativos o zero en la columna t_aculatamiento de un total de 731156
Hay 15508 valores negativos o zero en la columna t_carga de un total de 731156
Hay 11738 valores negativos o zero en la columna t_descarga de un total de 731156
Hay 405909 valores negativos o zero en la columna t_espera_pala de un total de 731156
Hay 363615 valores negativos o zero en la columna t_cola_camion de un total de 731156
Hay 0 valores negativos o zero en la columna t_a_pala de un total de 731156
Hay 0 valores negativos o zero en la columna t_viaje_descarga de un total de 731156
Hay 3366 valores negativos o zero en la columna t_ciclo_caex de un total de 731156
Hay 15290 valores negativos o zero en la columna t_ciclo_pala de un total de 731156
Hay 0 valores negativos o zero en la columna d_viaje_vacio de un total de 731156
Hay 0 valores negativ

Hay una gran cantidad de valores menores o iguales a 0 en la columna `t_espera_pala` y `t_cola_camion` por lo que removeremos estas columnas momentaneamente
### Eliminamos las filas que contengan valores negativos y los valores nulos

In [20]:
movimiento_mina = (
    movimiento_mina.select('*')
    .where((F.col('ton_camion') > 0)
          & (F.col('t_descarga') > 0)
          & (F.col('t_carga') > 0)
          & (F.col('t_ciclo_caex') > 0)
          & (F.col('t_ciclo_pala') > 0))
    .drop('t_espera_pala', 't_cola_camion', 't_aculatamiento', 't_a_pala')
    .na.drop()
)

Cantidad de `records` en la base de datos luego del filtrado

In [21]:
movimiento_mina.count()

567987

### Realizamos una agrupacion por hora de cada variable en `Datos_movimiento_mina.csv` para que cohincida con la base de datos `ads_produccion.csv` y podemos hacer un `join` de ambas

In [22]:
expression = [F.round(F.mean(col), 0).alias(col) for col in movimiento_mina.columns]
mov_historico_hora = (
    movimiento_mina.select('*')
    .groupBy(F.year(F.col('Dumping')).alias('y'), 
             F.month(F.col('Dumping')).alias('m'), 
             F.dayofmonth(F.col('Dumping')).alias('d'), 
             F.hour(F.col('Dumping')).alias('h'))
    .agg(*expression)
    .withColumn('tmp', F.concat(F.col("y"), F.lit("-"), F.col('m'), F.lit("-"), F.col('d'), F.lit(" "), F.col('h'), F.lit(':00:00')))
    .withColumn('date', F.to_timestamp(F.col('tmp')))
    .drop('y', 'm', 'd', 'h', 'tmp', 'Loading', 'Dumping')
    .orderBy('date')
)
mov_historico_hora.show(10, False)

+----------+--------+-------+----------+----------------+------------+------------+-------------+-------------+----------+----------+-------------------+
|ton_camion|n_baldes|t_carga|t_descarga|t_viaje_descarga|t_ciclo_caex|t_ciclo_pala|d_viaje_vacio|d_viaje_lleno|d_eq_vacio|d_eq_lleno|date               |
+----------+--------+-------+----------+----------------+------------+------------+-------------+-------------+----------+----------+-------------------+
|299.0     |4.0     |94.0   |64.0      |850.0           |1195.0      |147.0       |6111.0       |4157.0       |7991.0    |6533.0    |2020-12-31 21:00:00|
|307.0     |4.0     |106.0  |61.0      |937.0           |1764.0      |149.0       |5187.0       |4609.0       |7101.0    |7347.0    |2020-12-31 22:00:00|
|307.0     |4.0     |107.0  |66.0      |1038.0          |1998.0      |148.0       |5429.0       |5075.0       |7252.0    |8218.0    |2020-12-31 23:00:00|
|307.0     |4.0     |101.0  |57.0      |1074.0          |2117.0      |186.0 

Cantidad de `records` luego de realizado la grupacion por hora

In [23]:
mov_historico_hora.count()

12079

#### Visualizamos el tonelaje por camion versus el tiempo de ciclo de PALA, para un rango de fecha, donde hay 24 datos por dia

In [24]:
show(plot_rango_fecha(df=mov_historico_hora, date_column='date', x='t_ciclo_pala', y='ton_camion', fecha_inicio='2021-03-05', span=True, fecha_termino='2021-03-15',
                      extra_info=['n_baldes', 't_descarga'], cmap=False))

## Unimos ambas bases de datos en una sola, donde podremos extraer datos globales por hora, diarios, mensuales y anuales si queremos

### El JOIN lo haremos `inner` pero se van a perder 8100 filas de no datos (correspondientes a 2020), que despues se pueden compensar con alguna actualizacion de las bases de datos

In [25]:
global_por_hora = ads_produccion.join(mov_historico_hora, on=(ads_produccion['fecha'] == mov_historico_hora['date']), how='inner').drop('date', 'h')

Cantidad de `records` luego de realizado el `join` de ambas bases de datos

In [26]:
global_por_hora.count()

11771

Cantidad de variables en la nueva base de datos

In [27]:
global_por_hora.printSchema()

root
 |-- ton_total: double (nullable = true)
 |-- n_descargas: double (nullable = true)
 |-- n_cam: double (nullable = true)
 |-- n_shov: double (nullable = true)
 |-- ton_chancador: double (nullable = true)
 |-- ton_botadero: double (nullable = true)
 |-- descargas_botadero: double (nullable = true)
 |-- ton_chancador_1: double (nullable = true)
 |-- ton_chancador_2: double (nullable = true)
 |-- cam_chancador: double (nullable = true)
 |-- cam_botadero: double (nullable = true)
 |-- ton_alta_ley: double (nullable = true)
 |-- ton_media_ley: double (nullable = true)
 |-- ton_baja_ley: double (nullable = true)
 |-- ton_lastre: double (nullable = true)
 |-- fecha: timestamp (nullable = true)
 |-- ton_camion: double (nullable = true)
 |-- n_baldes: double (nullable = true)
 |-- t_carga: double (nullable = true)
 |-- t_descarga: double (nullable = true)
 |-- t_viaje_descarga: double (nullable = true)
 |-- t_ciclo_caex: double (nullable = true)
 |-- t_ciclo_pala: double (nullable = true)


### Calcular valores outliers para filtrar la base de datos `global_por_hora`

Creamos la variable `bounds` que calcula el `q1`, `q2`, `q3` y los valores minimos y maximos para poder realizar diagramas de cajas

In [28]:
bounds = {
    c: dict(zip(["q1", 'q2', "q3"], global_por_hora.approxQuantile(c, [0.25, 0.5, 0.75], 0))) for c, d in zip(global_por_hora.columns, global_por_hora.dtypes) if d[1] == "double"
}
for c in bounds:
    iqr = bounds[c]['q3'] - bounds[c]['q1']
    bounds[c]['q1'] = np.round(bounds[c]['q1'], 2)
    bounds[c]['q3'] = np.round(bounds[c]['q3'], 2)
    bounds[c]['min'] = np.round(bounds[c]['q1'] - (iqr * 1.5), 2)
    bounds[c]['max'] = np.round(bounds[c]['q3'] + (iqr * 1.5), 2)

In [29]:
def outlier_values(df, col, bounds):
    return np.unique(df.select(col)
            .where((F.col(col) < bounds[col]['min'])
                 | (F.col(col) > bounds[col]['max']))
            .rdd
            .flatMap(lambda x: x)
            .collect())

def outliers(df, cols=[], all_cols=False):
    out_dict = dict()
    if all_cols:
        for col in df.columns:
            try:
                out_dict[col] = outlier_values(df, col, bounds=bounds)
            except:
                continue
    else:
        for col in cols:
            out_dict[col] = outlier_values(df, col, bounds=bounds)
    return out_dict

In [30]:
dict_outliers_df = outliers(df=global_por_hora, cols=[], all_cols=True)

### Graficos de cajas de las distintas variables

In [31]:
def box_plot_mina(bounds, col, out_dict):
    df = pd.DataFrame(bounds).T
    p = figure(width = 350, height=350, tools="", background_fill_color="#efefef", x_range=[col], toolbar_location='above')

    p.segment([col], df.loc[col]['max'], [col], df.loc[col].q3, line_color="black")
    p.segment([col], df.loc[col]['min'], [col], df.loc[col].q1, line_color="black")

    # boxes
    p.vbar([col], 0.3, df.loc[col].q2, df.loc[col].q3, fill_color="#E08E79", line_color="black")
    p.vbar([col], 0.3, df.loc[col].q1, df.loc[col].q2, fill_color="#3B8686", line_color="black")

    # whiskers (almost-0 height rects simpler than segments)
    p.rect([col], df.loc[col]['max'], 0.1, 0.01, line_color="black")
    p.rect([col], df.loc[col]['min'], 0.1, 0.01, line_color="black")

    if not out_dict[col].size == 0:
        p.circle([col]*len(out_dict[col]), out_dict[col], size=6, color="#F38630", fill_alpha=0.6)

    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = "white"
    p.grid.grid_line_width = 2
    p.xaxis.major_label_text_font_size="14px"

    return p

In [32]:
s1 = box_plot_mina(bounds=bounds, col='ton_total', out_dict=dict_outliers_df)
s2 = box_plot_mina(bounds=bounds, col='n_descargas', out_dict=dict_outliers_df)
s3 = box_plot_mina(bounds=bounds, col='n_shov', out_dict=dict_outliers_df)
s4 = box_plot_mina(bounds=bounds, col='t_carga', out_dict=dict_outliers_df)
s5 = box_plot_mina(bounds=bounds, col='t_descarga', out_dict=dict_outliers_df)
s6 = box_plot_mina(bounds=bounds, col='d_viaje_vacio', out_dict=dict_outliers_df)
s7 = box_plot_mina(bounds=bounds, col='d_viaje_lleno', out_dict=dict_outliers_df)
s8 = box_plot_mina(bounds=bounds, col='t_ciclo_caex', out_dict=dict_outliers_df)
s9 = box_plot_mina(bounds=bounds, col='t_ciclo_pala', out_dict=dict_outliers_df)

grid = gridplot([[s1, s2, s3], [s4, s5, s6], [s7, s8, s9]])
show(grid)

Apesar del filtrado que hicimos anteriormente, la cantidad de `outliers` para algunas variables es muy grande, por lo que necesitamos filtrar estas variables y crear una nueva base de datos que tenga valores dentro de los rangos intercuartiles

#### Filtro la base de datos mediante un filtro que elimine los `outliers` en la base de datos `global_por_hora`

In [33]:
df = pd.DataFrame(bounds).T

In [34]:
for col in global_por_hora.columns:
    if col != 'fecha':
        global_por_hora = (
            global_por_hora.select('*')
            .where(F.col(col).between(df.loc[col]['min'], df.loc[col]['max']))
        )

Luego de eliminar los outliers, esta es la cantidad de `records` que quedan

In [35]:
global_por_hora.count()

5606

Volvemos a graficar una variable respecto del tonelaje total, luego de filtrar los valores `outliers `

In [103]:
show(plot_rango_fecha(df=global_por_hora, date_column='fecha', x='t_ciclo_pala', y='ton_total', fecha_inicio='2021-03-06', span=True, fecha_termino='2021-05-15',
                      extra_info=['n_baldes', 't_descarga'], cmap=False))

In [39]:
print(f'El rango de fechas de la nueva base de datos es desde {global_por_hora.agg(F.min(F.col("fecha"))).collect()[0][0]} hasta {global_por_hora.agg(F.max(F.col("fecha"))).collect()[0][0]}')

El rango de fechas de la nueva base de datos es desde 2020-12-31 21:00:00 hasta 2022-05-11 20:00:00


### Hacemos una agrupacion global por dia de la nueva base de datos `global_por_hora` para hacer analisis diarios

In [40]:
expression = [F.round(F.mean(col), 0).alias(col) for col in global_por_hora.columns]
global_por_dia = (
    global_por_hora.select('*')
    .groupBy(F.year(F.col('fecha')).alias('y'), 
             F.month(F.col('fecha')).alias('m'), 
             F.dayofmonth(F.col('fecha')).alias('d')) 
    .agg(*expression)
    .withColumn('tmp', F.concat(F.col("y"), F.lit("-"), F.col('m'), F.lit("-"), F.col('d')))
    .withColumn('date', F.to_date(F.col('tmp')))
    .drop('y', 'm', 'd', 'tmp', 'fecha')
    .orderBy('date')
)
global_por_dia.select('date', 'ton_total', 'n_cam', 'n_shov').show(10, False)

+----------+---------+-----+------+
|date      |ton_total|n_cam|n_shov|
+----------+---------+-----+------+
|2020-12-31|22155.0  |46.0 |6.0   |
|2021-01-01|18888.0  |39.0 |6.0   |
|2021-01-02|20964.0  |42.0 |6.0   |
|2021-01-03|18782.0  |42.0 |7.0   |
|2021-01-04|15044.0  |37.0 |6.0   |
|2021-01-05|16669.0  |40.0 |5.0   |
|2021-01-06|16704.0  |41.0 |6.0   |
|2021-01-07|17748.0  |42.0 |6.0   |
|2021-01-08|18421.0  |42.0 |7.0   |
|2021-01-09|19078.0  |42.0 |7.0   |
+----------+---------+-----+------+
only showing top 10 rows



### Esta funcion permite visualizar dos variables, en un rango de fechas indicado de manera diaria. Además podemos incorporar otra informacion de cualquier variable para complementar el análisis

In [41]:
def plot_pares_global_dia(df, x, y, date_column, fecha_inicio, fecha_termino, x_label='', y_label='', extra_info=[], span=True):
    """
    `df`  : pyspark DataFrame que contenga resultados de todas las variables agrupados por hora
    `x, y`: Nombre de columnas en `df` que se van a visualizar en el grafico
    `fecha_inicio`: fecha en formato yyyy-MM-dd que indica la fecha de inicio
    `fecha_inicio`: fecha en formato yyyy-MM-dd que indica la fecha de termino
    `date_column`: Nombre de columna en `df` que contenga Hora
    `x_label, `y_label`: Nombre de los ejes en el grafico
    `extra_info`: Lista que contenga nombres de variables que se quiera conocer la informacion
    `span`: Muestra una grilla que divide el grafico en 4, respecto del max y min de cada variable, default True.
    """
    df = (
        df.select('*')
        .filter(F.col(date_column).between(datetime.datetime.strptime(f'{fecha_inicio}', '%Y-%m-%d'),
                                            datetime.datetime.strptime(f'{fecha_termino}', '%Y-%m-%d')))
    ).toPandas()
    title = f'Rango desde {fecha_inicio} a {fecha_termino}'
    x_plot = df[x]
    y_plot = df[y]

    data = {'x': x_plot, 'y': y_plot, 'date': df[date_column]}
    for idx, c in enumerate(extra_info):
        data[f'c{idx}'] = df[c]
    tooltips = [(date_column, '@date{%Y-%m-%d}'), (y, '@y{int}'), (x, '@x{int}')] + [(f'{c}', f'@c{idx}') for idx, c in enumerate(extra_info)]
    source = ColumnDataSource(data=data)
    hover_tool = HoverTool(tooltips=tooltips, formatters={'@date': 'datetime'})
    
    p = figure(width=500, height=450, x_range=(x_plot.min(), x_plot.max()), y_range=(y_plot.min(), y_plot.max()))
    p.circle(x = 'x', y = 'y', size=10, color='#2171b5', alpha=0.7, source=source)
    p.title.align = 'center'
    p.title.text = title
    p.title.text_font_size = '14pt'
    if (x_label == '' or y_label == ''):
        p.xaxis.axis_label = x
        p.yaxis.axis_label = y
    else:
        p.xaxis.axis_label = x_label
        p.yaxis.axis_label = y_label
        
    if span:
        vline = Span(location=(x_plot.max()+x_plot.min())*0.5, dimension='height', line_color='black', line_width=3, line_alpha=0.7)
        hline = Span(location=(y_plot.max()+y_plot.min())*0.5, dimension='width', line_color='black', line_width=3, line_alpha=0.7)
        p.renderers.extend([vline, hline])
    p.add_tools(hover_tool)

    return p

#### Visualizamos multiples combinaciones de variables con el tonelaje total, para poder analizar su comportamiento

In [43]:
s1 = plot_pares_global_dia(df=global_por_dia, x='n_cam', y='ton_total', date_column='date', fecha_inicio='2021-01-01', fecha_termino='2022-05-01', span=True)
s2 = plot_pares_global_dia(df=global_por_dia, x='n_descargas', y='ton_total', date_column='date', fecha_inicio='2021-01-01', fecha_termino='2022-05-01', span=True)
s3 = plot_pares_global_dia(df=global_por_dia, x='ton_media_ley', y='ton_total', date_column='date', fecha_inicio='2021-01-01', fecha_termino='2022-05-01', span=True)
s4 = plot_pares_global_dia(df=global_por_dia, x='t_carga', y='ton_total', date_column='date', fecha_inicio='2021-01-01', fecha_termino='2022-05-01', span=True)
s5 = plot_pares_global_dia(df=global_por_dia, x='t_descarga', y='ton_total', date_column='date', fecha_inicio='2021-01-01', fecha_termino='2022-05-01', span=True)
s6 = plot_pares_global_dia(df=global_por_dia, x='d_viaje_vacio', y='ton_total', date_column='date', fecha_inicio='2021-01-01', fecha_termino='2022-05-01', span=True)
s7 = plot_pares_global_dia(df=global_por_dia, x='d_viaje_lleno', y='ton_total', date_column='date', fecha_inicio='2021-01-01', fecha_termino='2022-05-01', span=True)
s8 = plot_pares_global_dia(df=global_por_dia, x='t_ciclo_caex', y='ton_total', date_column='date', fecha_inicio='2021-01-01', fecha_termino='2022-05-01', span=True)
s9 = plot_pares_global_dia(df=global_por_dia, x='t_ciclo_pala', y='ton_total', date_column='date', fecha_inicio='2021-01-01', fecha_termino='2022-05-01', span=True)

grid = gridplot([[s1, s2, s3], [s4, s5, s6], [s7, s8, s9]], width=500, height=450)
show(grid)

In [44]:
s1 = plot_pares_global_dia(df=global_por_dia, x='n_shov', y='ton_total', date_column='date', fecha_inicio='2021-01-01', fecha_termino='2022-05-01', span=True)
s2 = plot_pares_global_dia(df=global_por_dia, x='cam_chancador', y='ton_total', date_column='date', fecha_inicio='2021-01-01', fecha_termino='2022-05-01', span=True)
s3 = plot_pares_global_dia(df=global_por_dia, x='cam_botadero', y='ton_total', date_column='date', fecha_inicio='2021-01-01', fecha_termino='2022-05-01', span=True)
s4 = plot_pares_global_dia(df=global_por_dia, x='descargas_botadero', y='ton_total', date_column='date', fecha_inicio='2021-01-01', fecha_termino='2022-05-01', span=True)
s5 = plot_pares_global_dia(df=global_por_dia, x='ton_lastre', y='ton_total', date_column='date', fecha_inicio='2021-01-01', fecha_termino='2022-05-01', span=True)
s6 = plot_pares_global_dia(df=global_por_dia, x='t_viaje_descarga', y='ton_total', date_column='date', fecha_inicio='2021-01-01', fecha_termino='2022-05-01', span=True)


grid = gridplot([[s1, s2, s3], [s4, s5, s6]], width=500, height=450)
show(grid)

### Creamos la base `tiempos_mina_global_hora` que agrupa de manera global las variables por rango de horas del dia, para identificar si la hora de día influye en el comportamiento de las variables

In [45]:
expression = [F.round(F.mean(col), 0).alias(col) for col in global_por_hora.columns]
tiempos_mina_global_hora = (
    global_por_hora.select('*')
    .groupBy(F.hour(F.col('fecha')).alias('Time'))
    .agg(*expression)
    .drop('fecha')
    .orderBy('Time')
)
tiempos_mina_global_hora.toPandas().head(10)

,Time,ton_total,n_descargas,n_cam,n_shov,ton_chancador,ton_botadero,descargas_botadero,ton_chancador_1,ton_chancador_2,...,n_baldes,t_carga,t_descarga,t_viaje_descarga,t_ciclo_caex,t_ciclo_pala,d_viaje_vacio,d_viaje_lleno,d_eq_vacio,d_eq_lleno
0,0,13572.0,44.0,37.0,7.0,5134.0,8438.0,28.0,2591.0,2543.0,...,4.0,124.0,62.0,987.0,1970.0,187.0,5260.0,4735.0,7995.0,8200.0
1,1,21261.0,69.0,45.0,7.0,8130.0,13131.0,43.0,4059.0,4071.0,...,4.0,123.0,64.0,982.0,2025.0,187.0,5404.0,4843.0,8249.0,8359.0
2,2,21271.0,69.0,45.0,7.0,8048.0,13223.0,43.0,4037.0,4011.0,...,4.0,125.0,63.0,976.0,1963.0,187.0,5377.0,4824.0,8120.0,8331.0
3,3,20882.0,68.0,45.0,7.0,7941.0,12941.0,42.0,4050.0,3890.0,...,4.0,127.0,64.0,982.0,1961.0,190.0,5304.0,4819.0,7984.0,8364.0
4,4,8521.0,28.0,24.0,6.0,3338.0,5183.0,17.0,1893.0,1444.0,...,4.0,124.0,63.0,1013.0,2004.0,191.0,5124.0,4599.0,7530.0,7733.0
5,5,18367.0,60.0,41.0,7.0,6999.0,11368.0,37.0,3584.0,3415.0,...,4.0,124.0,64.0,966.0,2066.0,188.0,5302.0,4705.0,8139.0,8147.0
6,6,20066.0,65.0,43.0,7.0,7505.0,12562.0,41.0,3825.0,3680.0,...,4.0,127.0,64.0,965.0,1992.0,191.0,5324.0,4772.0,8127.0,8207.0
7,7,21484.0,70.0,45.0,7.0,8058.0,13426.0,44.0,4061.0,3997.0,...,4.0,126.0,63.0,958.0,1956.0,190.0,5335.0,4771.0,7959.0,8122.0
8,8,9028.0,30.0,26.0,6.0,3330.0,5698.0,19.0,1664.0,1667.0,...,4.0,128.0,63.0,936.0,1893.0,192.0,5249.0,4797.0,7853.0,8187.0
9,9,15789.0,51.0,39.0,6.0,6504.0,9285.0,30.0,3398.0,3106.0,...,4.0,120.0,64.0,835.0,1258.0,184.0,5073.0,4406.0,8002.0,7392.0


#### Esta funcion permite graficar el DataFrame `tiempos_mina_global_hora` comparando todas las variables respecto al tonelaje, donde tambien se puede añadir informacion extra con `extra_info`

In [46]:
def plot_pares_global_hora(df, x, y, date_column, x_label='', y_label='', extra_info=[], span=True):
    """
    `df`  : pyspark DataFrame que contenga resultados de todas las variables agrupados por hora
    `x, y`: Nombre de columnas en `df` que se van a visualizar en el grafico
    `date_column`: Nombre de columna en `df` que contenga Hora
    `x_label, `y_label`: Nombre de los ejes en el grafico
    `extra_info`: Lista que contenga nombres de variables que se quiera conocer la informacion
    `span`: Muestra una grilla que divide el grafico en 4, respecto del max y min de cada variable, default True.
    """
    df = df.toPandas()
    x_plot = df[x]
    y_plot = df[y]
    data = {'x': x_plot, 'y': y_plot, 'date': df[date_column]}
    for idx, c in enumerate(extra_info):
        data[f'c{idx}'] = df[c]
    tooltips = [(date_column, f'@date:00 hrs'), (y, '@y'), (x, '@x')] + [(f'{c}', f'@c{idx}') for idx, c in enumerate(extra_info)]
    
    source = ColumnDataSource(data=data)
    hover_tool = HoverTool(tooltips=tooltips)
    mapper = linear_cmap(field_name='date', palette=all_palettes['PiYG'][4], low=min(df[date_column]), high=max(df[date_column]))
    color_bar = ColorBar(color_mapper=mapper['transform'], width = 20, ticker=FixedTicker(ticks=np.arange(0, 36, 6), desired_num_ticks=5), scale_alpha=0.7)
    
    p = figure(width=525, height=450, x_range=(x_plot.min(), x_plot.max()), y_range=(y_plot.min(), y_plot.max()))
    p.circle(x = 'x', y = 'y', size=25, color=mapper, alpha=0.7, source=source)
    p.title.align = 'center'
    p.title.text = f'Grafico {y_label} vs {x_label}'
    p.title.text_font_size = '12pt'
    if (x_label == '' or y_label == ''):
        p.xaxis.axis_label = x
        p.yaxis.axis_label = y
    else:
        p.xaxis.axis_label = x_label
        p.yaxis.axis_label = y_label

    vline = Span(location=(x_plot.max()+x_plot.min())*0.5, dimension='height', line_color='black', line_width=3, line_alpha=0.6)
    hline = Span(location=(y_plot.max()+y_plot.min())*0.5, dimension='width', line_color='black', line_width=3, line_alpha=0.6)
    if span:
        p.renderers.extend([vline, hline])
    p.add_tools(hover_tool)
    p.add_layout(color_bar, 'right')
    
    return p

#### Realizamos una visualizacion de muchas variables respecto al tonelaje total

In [47]:
s1 = plot_pares_global_hora(df=tiempos_mina_global_hora, date_column='Time', x='n_cam', y='ton_total', x_label='Numero Camiones', y_label='Tonelaje', span=True)
s2 = plot_pares_global_hora(df=tiempos_mina_global_hora, date_column='Time', x='n_descargas', y='ton_total', x_label='Numero Descargas', y_label='Tonelaje', span=True)
s3 = plot_pares_global_hora(df=tiempos_mina_global_hora, date_column='Time', x='ton_media_ley', y='ton_total', x_label='Tonelaje Ley Media', y_label='Tonelaje', span=True)
s4 = plot_pares_global_hora(df=tiempos_mina_global_hora, date_column='Time', x='t_carga', y='ton_total', x_label='Tiempo carga', y_label='Tonelaje', span=True)
s5 = plot_pares_global_hora(df=tiempos_mina_global_hora, date_column='Time', x='t_descarga', y='ton_total', x_label='Tiempo descarga', y_label='Tonelaje', span=True)
s6 = plot_pares_global_hora(df=tiempos_mina_global_hora, date_column='Time', x='d_viaje_vacio', y='ton_total', x_label='Distancia viaje vacio', y_label='Tonelaje', span=True)
s7 = plot_pares_global_hora(df=tiempos_mina_global_hora, date_column='Time', x='d_viaje_lleno', y='ton_total', x_label='Distancia viaje lleno', y_label='Tonelaje', span=True)
s8 = plot_pares_global_hora(df=tiempos_mina_global_hora, date_column='Time', x='t_ciclo_caex', y='ton_total', x_label='Tiempo ciclo CAEX', y_label='Tonelaje', span=True)
s9 = plot_pares_global_hora(df=tiempos_mina_global_hora, date_column='Time', x='t_ciclo_pala', y='ton_total', x_label='Tiempo ciclo PALA', y_label='Tonelaje', span=True)

grid = gridplot([[s1, s2, s3], [s4, s5, s6], [s7, s8, s9]], width=500, height=450)
show(grid)

In [48]:
s1 = plot_pares_global_hora(df=tiempos_mina_global_hora, date_column='Time', x='n_shov', y='ton_total', x_label='Numero Palas', y_label='Tonelaje', span=True)
s2 = plot_pares_global_hora(df=tiempos_mina_global_hora, date_column='Time', x='cam_chancador', y='ton_total', x_label='Camiones a chancador', y_label='Tonelaje', span=True)
s3 = plot_pares_global_hora(df=tiempos_mina_global_hora, date_column='Time', x='cam_botadero', y='ton_total', x_label='Camiones a botadero', y_label='Tonelaje', span=True)
s4 = plot_pares_global_hora(df=tiempos_mina_global_hora, date_column='Time', x='descargas_botadero', y='ton_total', x_label='Descargas a botadero', y_label='Tonelaje', span=True)
s5 = plot_pares_global_hora(df=tiempos_mina_global_hora, date_column='Time', x='ton_lastre', y='ton_total', x_label='Tonelaje de lastre', y_label='Tonelaje', span=True)
s6 = plot_pares_global_hora(df=tiempos_mina_global_hora, date_column='Time', x='t_viaje_descarga', y='ton_total', x_label='Tiempo viaje descargas', y_label='Tonelaje', span=True)

grid = gridplot([[s1, s2, s3], [s4, s5, s6]], width=500, height=450)
show(grid)

## Analisis estadistico
### Histograma variables a estudiar

In [100]:
def hist_mina(df, target='', bins=10, x_label=''):
    """
    `df`: Pyspark DataFrame
    `target`: Nombre de Columna que indica la variable a calcular su histograma
    `bins`: Numero de particiones para realizar el histograma
    `x_label`: Nombre del eje x en el grafico
    """
    df = df.toPandas()
    stats = np.round(df[target].describe(), 2)
    hist, edges = np.histogram(df[target], bins = bins)
    plot = figure(x_range=(min(edges)*0.9, max(edges)*1.07), y_range=(0, max(hist)*1.2), plot_width=500, plot_height=450, tools='')
    plot.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], fill_color='red', line_color='black')
    plot.xaxis.axis_label = x_label
    plot.yaxis.axis_label = 'Frecuencia'
    
    text = f'Media:   {stats["mean"]}\nMinimo: {stats["min"]}\nMaximo: {stats["max"]}'
    plot.text(x=[stats['min']*1.01], y =[max(hist)], text=[text], angle = 0)

    return plot

In [101]:
s1 = hist_mina(global_por_hora, target='n_cam', bins=10, x_label='Numero de camiones')
s2 = hist_mina(global_por_hora, target='n_descargas', bins=10, x_label='Numero de descargas')
s3 = hist_mina(global_por_hora, target='n_shov', bins=5, x_label='Numero de palas')
s4 = hist_mina(global_por_hora, target='t_carga', bins=10, x_label='Tiempo de carga')
s5 = hist_mina(global_por_hora, target='t_descarga', bins=10, x_label='Tiempo de descarga')
s6 = hist_mina(global_por_hora, target='d_viaje_vacio', bins=10, x_label='Distancia viaje vacio')
s7 = hist_mina(global_por_hora, target='d_viaje_lleno', bins=10, x_label='Distancia viaje lleno')
s8 = hist_mina(global_por_hora, target='t_ciclo_caex', bins=10, x_label='Tiempo ciclo CAEX')
s9 = hist_mina(global_por_hora, target='t_ciclo_pala', bins=10, x_label='Tiempo ciclo PALA')

grid = gridplot([[s1, s2, s3], [s4, s5, s6], [s7, s8, s9]], width=500, height=450)
show(grid)

In [102]:
s1 = hist_mina(global_por_hora, target='n_shov', bins=10, x_label='Numero de palas')
s2 = hist_mina(global_por_hora, target='cam_chancador', bins=10, x_label='Camiones a chancador')
s3 = hist_mina(global_por_hora, target='cam_botadero', bins=10, x_label='Camiones a botadero')
s4 = hist_mina(global_por_hora, target='descargas_botadero', bins=10, x_label='Descargas a botadero')
s5 = hist_mina(global_por_hora, target='ton_lastre', bins=10, x_label='Toneladas de lastre')
s6 = hist_mina(global_por_hora, target='t_viaje_descarga', bins=10, x_label='Tiempo viaje descargas')

grid = gridplot([[s1, s2, s3], [s4, s5, s6]], width=500, height=450)
show(grid)

Podemos observar que practicamente todas las variables presentan un comportamiento normal

#### Graficos de barras de las distintas variables, respecto de la hora del día

In [104]:
df = tiempos_mina_global_hora.toPandas()
source_available = ColumnDataSource(df)
source_visible = ColumnDataSource(data = dict(x = df['Time'], y = df['t_ciclo_caex']))

p = figure(x_range=(df['Time'].min()-0.5, df['Time'].max()+0.5), 
                  plot_width=700, plot_height=450, x_axis_type=None)
p.vbar(x='x', bottom = 0, top='y', source=source_visible, width=0.75)
ticker = SingleIntervalTicker(interval=1, num_minor_ticks=1)
xaxis = LinearAxis(ticker=ticker)
p.add_layout(xaxis, 'below')
p.xaxis.axis_label = 'Hora del dia'
p.yaxis.axis_label = 'Valor promedio por hora'
p.title.text = 'Grafico agrupado respecto a la hora del dia'
p.title.text_font_size = '14pt'
p.title.align = 'center'

js_code = """
    var sli1 = select.value;
    var data_visible = source_visible.data;
    var data_available = source_available.data;
    var y_label = sli1.toString();
    data_visible.y = data_available[sli1.toString()];
    source_visible.change.emit();
"""

select = Select(title = "Variable", value = 't_ciclo_caex', options=list(df.columns))
select.js_on_change('value', CustomJS(args=dict(source_visible = source_visible, 
                                                source_available = source_available,
                                                select = select), 
                                      code=js_code))

show(column(select, p))

En la variable `t_ciclo_caex` se observa como a las `9:00 hr` y `21:00 hr` hay una baja en el tiempo de ciclo debido a que los turnos en la mina son de 12 horas y se producen a esas mismas horas

Para el caso de `ton_total` observamos que presenta un comportamiento ciclico, al igual que para la variable `n_descargas`, `n_cam` entre otras variables